Miguel Gutierrez y Daniel Rambaut

# Proyecto IV: Chatbot

EL objetivo de este poryecto es desarrollar un sistema de dialogo tipo **chatbot** utilizando las herramientas vistas en clase. EL sistema debe contener un modeulo de atención, el cual incluira información historica del chat en uno o más vectores caracteristicos, junto con la codificación de la palabra escpecificada. Ademas se hará uso de una red LSTM para la generación de texto, de forma similar a como se hizó en el proyecto pasado. 

Para el desarrollo de este proyecto deben seguir lso siguientes pasos:

1. Crear el corpus de entrenamiento. Esto se hace en conjunto con los demás grupos del curso.
2. Diseñar la estrategia de atención, aqui ustedes definen el número de vectores historicos qu eutilian, como los calculan, la longitud del historial, etc...
3. Deben crear el sistema de generación de texto, el cual permita establecer un dialogo con el computador.
4. Deben organizar el código de tal forma que permit ala interacción con el y se pueda establecer una comunicación adecuada. Es decir, deben generar una función que reciba como entrada una frase, que la respuesta sea la generación de texto del chatbot, y que al colocarle la siguiente frase de entrada, se almacene en el historial la conversación que se lleva hasta el momento, para que el sistema pueda responder de forma adecuada.

Al finalizar la implementación deben responder las siguientes preguntas:

1. ¿Qué diferencias encuentran en el sistema de generación de texto de este proyecto, comparado con el del proyecto anterior?
2. ¿Considera qu ela estrategia de atención mejora el rendimiento del sistema de generación de texto? Justifique su respuesta.
3. ¿Son los resultados obtenidos satisfactorios? ¿Cómo mejoraria los resultados del sistema implementado?
4. Si se les solicitará en sus trabajos utilizar los conocimientos que adquirieron en NLP para proponer propuestas de desarrollos y mejorar la competitividad de sus empresas, ¿Qué proyectos propondrian?
5. Esta pregunta pueden o no contestarla, quisiera saber la opinión al respecto del curso, que les gusto, que no les gusto, que cambiarian.

Recuerden que la fecha de entrega de este proyecto es el **Viernes 28 de Mayo, 2021, a las 12 de la noche.**

Muchas gracias por la atención que prestarón en el grupo y su interes en las clases. 

In [1]:
import re 
import numpy as np
from os.path import join
import json
from tensorflow.keras import models,layers,optimizers, callbacks, regularizers
from scipy.spatial.distance import cdist
from scipy.special import softmax

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding,Conv2D,MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt

Verificamos que vamos a utilizar procesador grafico, para correr las redes mas rapido.

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


## Lectura Corpus

In [3]:
def get_file_data(fname):
    file_contents = []
    with open(fname, encoding='utf8',errors='ignore') as f:
        file_contents = f.read()
    text = []
    for val in file_contents.split('\n'):
        val = re.sub(r'á','a',val)
        val = re.sub(r'é','e',val)
        val = re.sub(r'í','i',val)
        val = re.sub(r'ó','o',val)
        val = re.sub(r'ú','u',val)
        val = re.sub(r'ü','u',val)
        val = re.sub(r'Á','A',val)
        val = re.sub(r'É','E',val)
        val = re.sub(r'Í','I',val)
        val = re.sub(r'Ó','O',val)
        val = re.sub(r'Ú','U',val)
        val = re.sub(r'ñ','n',val)
        val = re.sub(r'Ñ','N',val)
        sent = re.findall(r"[A-Za-z]+|[.,!?;:¿¡]", val)
        line = ''
        for words in sent:
            if len(words) >= 1 :
                line += ' ' + words
        text.append(line)
    return text

# Función para obtener un vocabulario en función del texto procesado

def generate_dictionary_data(text):
    word_to_index= dict()
    index_to_word = dict()
    corpus = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in re.findall(r"[A-Za-z]+|[.,!?;:¿¡]", row):
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index, index_to_word, corpus, vocab_size, length_of_corpus

# Función para generar representaciones one hot de los vectores target y del corpus

def get_one_hot_vectors(target_word, context_words, vocab_size, word_to_index):
    
    #Create an array of size = vocab_size filled with zeros
    trgt_word_vector = np.zeros(vocab_size)
    
    #Get the index of the target_word according to the dictionary word_to_index. 
    index_of_word_dictionary = word_to_index.get(target_word) 
    
    #Set the index to 1
    trgt_word_vector[index_of_word_dictionary] = 1
    
    #Repeat same steps for context_words but in a loop
    ctxt_word_vector = np.zeros(vocab_size)
    
    
    for word in context_words:
        index_of_word_dictionary = word_to_index.get(word) 
        ctxt_word_vector[index_of_word_dictionary] = 1
        
    return trgt_word_vector, ctxt_word_vector

# Función para generar los datos de entrenamiento para la red neuronal que representa el modelo word2vec

def generate_training_data_word2vec(corpus, window_size, vocab_size, word_to_index, length_of_corpus):

    input_data = []
    output_data = []

    for i, word in enumerate(corpus):

        index_target_word = i
        target_word = word
        context_words = []

        #when target word is the first word
        if i == 0:  

            # trgt_word_index:(0), ctxt_word_index:(1,2)
            context_words = [corpus[x] for x in range(i + 1 , window_size + 1)] 


        #when target word is the last word
        elif i == len(corpus)-1:

            # trgt_word_index:(9), ctxt_word_index:(8,7), length_of_corpus = 10
            context_words = [corpus[x] for x in range(length_of_corpus - 2 ,length_of_corpus -2 - window_size  , -1 )]

        #When target word is the middle word
        else:

            #Before the middle target word
            before_target_word_index = index_target_word - 1
            for x in range(before_target_word_index, before_target_word_index - window_size , -1):
                if x >=0:
                    context_words.extend([corpus[x]])

            #After the middle target word
            after_target_word_index = index_target_word + 1
            for x in range(after_target_word_index, after_target_word_index + window_size):
                if x < len(corpus):
                    context_words.extend([corpus[x]])


        trgt_word_vector, ctxt_word_vector = get_one_hot_vectors(target_word, context_words, vocab_size, word_to_index)
        input_data.append(ctxt_word_vector)
        output_data.append(trgt_word_vector)
        
    return np.array(input_data), np.array(output_data)

def generate_training_data_LSTM1(corpus, word_to_index, window_size, vectors):
    vocab_size, N = vectors.shape

    input_data = []
    output_data = []

    for i, word in enumerate(corpus[window_size:]):
        ctxt_words = corpus[i:window_size+i]
        ctxt_matrix = np.zeros((window_size, vectors.shape[0]))
        for j, ctxt in enumerate(ctxt_words):
            idx = word_to_index[ctxt]
            ctxt_matrix[j, :] = vectors[:, idx]
        idx = word_to_index[word]
        # words_vec = np.zeros(#pal)
        # word_vec[idx] = 1
        word_vec = vectors[:, idx]
        input_data.append(ctxt_matrix)
        output_data.append(word_vec)
    
    input_data = np.array(input_data)
    output_data = np.array(output_data)

    return input_data, output_data

def generate_training_data_LSTM2(corpus, word_to_index, window_size, vectors):
    vocab_size, N = vectors.shape

    input_data = []
    output_data = []

    for i, word in enumerate(corpus[window_size:]):
        ctxt_words = corpus[i:window_size+i]
        ctxt_matrix = np.zeros((window_size, vectors.shape[0]))
        for j, ctxt in enumerate(ctxt_words):
            idx = word_to_index[ctxt]
            ctxt_matrix[j, :] = vectors[:, idx]
        idx = word_to_index[word]
        word_vec = np.zeros(vectors.shape[1])
        word_vec[idx] = 1
        input_data.append(ctxt_matrix)
        output_data.append(word_vec)
    
    input_data = np.array(input_data)
    output_data = np.array(output_data)

    return input_data, output_data

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5171744789252252627
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 510377984
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16033786581469252523
physical_device_desc: "device: 0, name: NVIDIA GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [5]:
fname = "Conv1.txt"
C = 3 # Número de palabras de contexto a la derecha y a la izquierda
text = get_file_data(fname)
word_to_index,index_to_word,corpus,vocab_size,length_of_corpus = generate_dictionary_data(text)
N = 300
tipo = "chatbot"

In [6]:
vocab_size

323

## Word2vec

Generamos el embebimiento de las palabras.

In [8]:
input_data, output_data = generate_training_data_word2vec(corpus,C,vocab_size,word_to_index,length_of_corpus)

In [9]:
model = models.Sequential()
model.add(layers.Dense(N, activation = 'relu', input_shape = (vocab_size,)))
model.add(layers.Dense(vocab_size, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               97200     
_________________________________________________________________
dense_1 (Dense)              (None, 323)               97223     
Total params: 194,423
Trainable params: 194,423
Non-trainable params: 0
_________________________________________________________________


In [10]:
cbs = [callbacks.ModelCheckpoint(join("word2vec_{}.h5".format(tipo)), monitor='accuracy', save_best_only=True)]
history = model.fit(input_data, output_data, epochs = 20, verbose=1, callbacks=cbs)
history_dict = history.history
json.dump(history_dict, open(join('word2vec_{}.json'.format(tipo)), 'w')) 

Epoch 1/20
38/38 [==============================] - 2s 12ms/step - loss: 5.5153 - accuracy: 0.0955
Epoch 2/20
38/38 [==============================] - 0s 11ms/step - loss: 4.5422 - accuracy: 0.1512
Epoch 3/20
38/38 [==============================] - 0s 10ms/step - loss: 4.0545 - accuracy: 0.2309
Epoch 4/20
38/38 [==============================] - 0s 12ms/step - loss: 3.7014 - accuracy: 0.2865
Epoch 5/20
38/38 [==============================] - 0s 10ms/step - loss: 3.3574 - accuracy: 0.3056
Epoch 6/20
38/38 [==============================] - 0s 7ms/step - loss: 3.0241 - accuracy: 0.3248
Epoch 7/20
38/38 [==============================] - 0s 7ms/step - loss: 2.7030 - accuracy: 0.3621
Epoch 8/20
38/38 [==============================] - 0s 11ms/step - loss: 2.3830 - accuracy: 0.4385
Epoch 9/20
38/38 [==============================] - 0s 11ms/step - loss: 2.0628 - accuracy: 0.5199
Epoch 10/20
38/38 [==============================] - 0s 12ms/step - loss: 1.7593 - accuracy: 0.5914
Epoch 11/20

## Cargar el word2vec

In [6]:
model = models.load_model(join('word2vec_{}.h5'.format(tipo)))
w0 = model.get_weights()[2]
w1 = model.get_weights()[0]
vectors = 0.5*(w0+w1.T)
del model

## Crear modelo de atencion

Vamos a leer el texto y encontrar el Historial y Prediccion Y

In [8]:
def read_data(fname,vectors,len_word2index):
    file_contents = []
    with open(fname, encoding='utf8',errors='ignore') as f:
        file_contents = f.read()
    text = []
    general =[]
    onehot_context = []
    general_hot = []
    max_palabras =0
    for val in file_contents.split('\n'):
        contexto = []
        val = re.sub(r'á','a',val)
        val = re.sub(r'é','e',val)
        val = re.sub(r'í','i',val)
        val = re.sub(r'ó','o',val)
        val = re.sub(r'ú','u',val)
        val = re.sub(r'ü','u',val)
        val = re.sub(r'Á','A',val)
        val = re.sub(r'É','E',val)
        val = re.sub(r'Í','I',val)
        val = re.sub(r'Ó','O',val)
        val = re.sub(r'Ú','U',val)
        val = re.sub(r'ñ','n',val)
        val = re.sub(r'Ñ','N',val)
        va1 = re.sub(r'.','',val)
        sent = re.findall(r"[A-Za-z]+|[.,!?;:¿¡]", val)
        line = ''

        conteo = 0
        for words in sent:
            words = words.lower()
            if len(words) >= 1 :
                idx = word_to_index[words]
                vec_word = vectors[:,idx]
                contexto.append(vec_word)
                onehot_context.append(idx)
                line += ' ' + words
                conteo += 1
        if (max_palabras < conteo): max_palabras = conteo
        text.append(contexto)
        context = np.array(contexto)
        onehot_context = np.array(onehot_context)
        encoded = to_categorical(onehot_context, num_classes=len_word2index)
        general_hot.append(encoded)
        onehot_context = []
        general.append(contexto)
    return general,max_palabras,general_hot

In [9]:
general_contexto,max_palabras,general_hot = read_data(fname,vectors,len(word_to_index))


Creamos la funcion de obtener el O=\[ Q | H \]

In [10]:
def get_O(contexto,quest,mayor):
    quest1 = np.zeros((mayor,300))
    H = []
    for j in contexto:
        for k in j:
            H.append(k)

    H = np.array(H)

    for i in range(len(quest)):
        quest1[i,:] = quest[i]
    #quest = np.array(quest)

    #print("h shape: ",H.shape)
    #print(quest1.shape)

    QIT = quest1 @ H.T
    #print(QIT.shape)

    a = softmax(QIT)

    O = a @ H

    return O,quest1

In [13]:
contexto = general_contexto[0:3]
quest = general_contexto[3]

In [14]:
O,quest1 = get_O(contexto,quest,max_palabras)

In [22]:
quest1.shape

(31, 300)

In [12]:
def Q_O(contexto,quest,mayor):
    aux,quest1 = get_O(contexto,quest,mayor)
    q =  np.concatenate((quest1,aux),axis=1)
    return q,quest1

In [24]:
q,quest1 = Q_O(contexto,quest,max_palabras)
q.shape

(31, 600)

## Creacion Valores X y Y

Se escoge el tamaño de la historia previa que será lh =4

In [15]:
lh = 4

X = []
Y = []
# ONE HOT
print("entrenando...")
for i in range(0,len(general_contexto)-lh-1):
    H = general_contexto[i:lh+i]
    quest = general_contexto[lh+i]
    quest = np.array(quest)
    q,quest1 = Q_O(H,quest,max_palabras)
    X.append(q)
    questy = general_hot[lh+i+1]
    quest_o = np.zeros((max_palabras,vocab_size))
    for i in range(questy.shape[0]):
        quest_o[i,:] = questy[i]
    Y.append(quest_o)

print("termino...")

entrenando...
termino...


In [16]:
X = np.array(X)
Y = np.array(Y)

Creamos el modelo de prediccion utilizando atencion

In [28]:
model = Sequential()
model.add(LSTM(1000, input_shape=(X[0].shape),return_sequences=True,activity_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(layers.LSTM(800, activation='tanh',return_sequences=True,activity_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(500, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(layers.Dense(322, activation='sigmoid'))
model.add(Dense(Y[0].shape[1], activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 31, 1000)          6404000   
_________________________________________________________________
dropout (Dropout)            (None, 31, 1000)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 31, 800)           5763200   
_________________________________________________________________
dense_2 (Dense)              (None, 31, 500)           400500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 500)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 31, 322)           161322    
_________________________________________________________________
dense_4 (Dense)              (None, 31, 323)          

En caso de cargar el modelo con la mejor epoca (Ya que es muy variable en su accuracy). Recomendamos hacerlo

In [17]:
model = models.load_model('ModeloFinal_chatbot.h5')

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
cbs = [callbacks.ModelCheckpoint(join("ModeloFinal_chatbot.h5"), monitor='accuracy', save_best_only=True)]
history = model.fit(X, Y, epochs = 500, verbose=1, callbacks=cbs,batch_size=32,validation_split=0.2)

=============================] - 1s 225ms/step - loss: 1.4265 - accuracy: 0.0287 - val_loss: 1.9212 - val_accuracy: 0.0310
Epoch 362/500
4/4 [==============================] - 1s 224ms/step - loss: 1.4246 - accuracy: 0.0287 - val_loss: 1.9182 - val_accuracy: 0.0310
Epoch 363/500
4/4 [==============================] - 1s 229ms/step - loss: 1.4218 - accuracy: 0.0283 - val_loss: 1.9162 - val_accuracy: 0.6839
Epoch 364/500
4/4 [==============================] - 1s 226ms/step - loss: 1.4204 - accuracy: 0.7387 - val_loss: 1.9153 - val_accuracy: 0.6839
Epoch 365/500
4/4 [==============================] - 1s 224ms/step - loss: 1.4226 - accuracy: 0.7384 - val_loss: 1.9158 - val_accuracy: 0.6839
Epoch 366/500
4/4 [==============================] - 1s 225ms/step - loss: 1.4262 - accuracy: 0.7387 - val_loss: 1.9143 - val_accuracy: 0.6839
Epoch 367/500
4/4 [==============================] - 1s 225ms/step - loss: 1.4262 - accuracy: 0.7387 - val_loss: 1.9074 - val_accuracy: 0.6839
Epoch 368/500
4/4 [

In [18]:
def read_chat(corpus,vectors,len_word2index):
    file_contents = []
    text = []
    general =[]
    onehot_context = []
    general_hot = []
    max_palabras =0
    for val in corpus.split('\n'):
        contexto = []
        val = re.sub(r'á','a',val)
        val = re.sub(r'é','e',val)
        val = re.sub(r'í','i',val)
        val = re.sub(r'ó','o',val)
        val = re.sub(r'ú','u',val)
        val = re.sub(r'ü','u',val)
        val = re.sub(r'Á','A',val)
        val = re.sub(r'É','E',val)
        val = re.sub(r'Í','I',val)
        val = re.sub(r'Ó','O',val)
        val = re.sub(r'Ú','U',val)
        val = re.sub(r'ñ','n',val)
        val = re.sub(r'Ñ','N',val)
        va1 = re.sub(r'.','',val)
        sent = re.findall(r"[A-Za-z]+|[.,!?;:¿¡]", val)
        line = ''

        conteo = 0
        for words in sent:
            words = words.lower()
            if len(words) >= 1 :
                idx = word_to_index[words]
                vec_word = vectors[:,idx]
                contexto.append(vec_word)
                onehot_context.append(idx)
                line += ' ' + words
                conteo += 1
        if (max_palabras < conteo): max_palabras = conteo
        text.append(contexto)
        context = np.array(contexto)
        onehot_context = np.array(onehot_context)
        encoded = to_categorical(onehot_context, num_classes=len_word2index)
        general_hot.append(encoded)
        onehot_context = []
        general.append(contexto)
    return general,max_palabras,general_hot

## Funcion CHATBOT

In [27]:
def robot(oracion,corpus,pos,max_palabras=30):
    print("---------")
    X = []
    if pos != 0:
        oracion = "<start> "+oracion+" <eos>"
        corpus = corpus +'\n'
    else:
        oracion = "<start> "+oracion+" <eos>"
        corpus = oracion+"\n"+oracion+ "\n"+oracion+"\n"+oracion+"\n"
        corpus = corpus + oracion

    general_contexto,max_palabras2,general_hot = read_chat(corpus,vectors,len(word_to_index))

    H = general_contexto[pos:lh+pos]
    quest = general_contexto[lh+pos]


    quest = np.array(quest)
    q,quest1 = Q_O(H,quest,max_palabras)
    q = tf.expand_dims(q, axis=0)
    questy = (model.predict(q) > 0.2).astype("int32")
    indices = []
    for i in range(questy.shape[1]):
        ind = np.argmax(questy[0,i,:])
        if ind !=0:
            indices.append(ind)
    oracion = ''
    pos +=1
    for i in indices:
        oracion = oracion + ' '+index_to_word[i]
    oracion = "<start> "+oracion+" <eos>"
    print("Robot: ",oracion)
    corpus = corpus +'\n'+ oracion 
    return corpus,pos

## EJEMPLO Funcionamiento Chatbot

In [32]:
oracion = "<start> <begin> +'\n'\
Hola buenos días.+'\n' \
Hola ¿Como vas ? +'\n' \
bien bien y ¿tú? +'\n' \
bien bien ¿Qué haces?"
print(oracion)
corpus,pos = robot(oracion,'',0,max_palabras=31)
oracion = "También estoy hablando contigo. "
print(oracion)
corpus,pos = robot(oracion,corpus,pos,max_palabras=31)
oracion = "¿ Cuantos años tienes ?"
print(oracion)
corpus,pos = robot(oracion,corpus,pos,max_palabras=31)


<start> <begin> +'
'Hola buenos días.+'
' Hola ¿Como vas ? +'
' bien bien y ¿tú? +'
' bien bien ¿Qué haces?
---------
Robot:  <start>  hablando hablando contigo ¿ <eos>
También estoy hablando contigo. 
---------
Robot:  <start>  hola <eos>
¿ Cuantos años tienes ?
---------
Robot:  <start>  bien ¿ <eos>


## PREGUNTAS

1. ¿Qué diferencias encuentran en el sistema de generación de texto de este proyecto, comparado con el del proyecto anterior?

    * El sistema de generacion de texto tenia en cuenta un rango de contexto de palabras anteriores para generar una nueva palabra. En este problema ya no tenemos un conjunto de palabras, si no un conjunto de historia que corresponde a oraciones anteriores del Chat. Ademas se 'contrasta' la relacion de las lineas anteriores con la linea actual para entender el contexto del chat que es el que denominamos O para generar una nueva oracion.

2. ¿Considera qu ela estrategia de atención mejora el rendimiento del sistema de generación de texto? Justifique su respuesta.

    * Si, la estrategia atencion mejora el rendimiento del sistema puesto tiene en cuenta la relacion que tiene la oracion actual con las anteriores y que tanto a cada palabra de cada una, cosa que dificilmente captaria un modelo neuronal simple. Algo que encontramos interesantes que es FUNDAMENTAL poner en el texto <Start> y <end> al final de cada oracion para que la red pudiera mejorar su precision y generar mejores textos.

3. ¿Son los resultados obtenidos satisfactorios? ¿Cómo mejoraria los resultados del sistema implementado?
    * Los resultados no son satisfactorios bien por que la prediccion no necesariamente corresponde a una buena prediccion esto pues la cantidad de datos (tamaño de corpus) es muy pequeña, ademas que parece que el espacio de entrenamiento no es convexo donde aveces pasa de una accuracy de 0.75 a 0.02, lo cual es un problema para el entrenamiento de la red. Sin embargo, de cierta manera el robot si es capaz de generar una respuesta que tenga sentido y capta cada oracion de la conversacion original. En nuestro problema solo predijo bien la primera linea, ya despues comienza a fallar un poco, esto pues se añade ruido al robot no responde correctamente.

4. Si se les solicitará en sus trabajos utilizar los conocimientos que adquirieron en NLP para proponer propuestas de desarrollos y mejorar la competitividad de sus empresas, ¿Qué proyectos propondrian?

    * Creemos depende del sector de la empresa. Bien ya con los conocimientos que tenemos podemos generar soluciones de crear un Chatbot de la empresa. Entender textos y mapearlos a un embebimiento donde se puede ver relacione entre ellos. Asi como crear soluciones que pueda captar audio y transformalo a texto. Miguel actualmente en el trabajo se encuentra ayudando en un proyecto de analizar sesgos de genero en las postuaciones de trabajo utilizando NLP, donde ha podido ayudar al grupo de trabajo dado los conocimientos aprendidos en NLP.


5. Esta pregunta pueden o no contestarla, quisiera saber la opinión al respecto del curso, que les gusto, que no les gusto, que cambiarian.

    * Nos parecio un buen curso, que explica lo fundamental si despues queremos profundizar y meternos mas de relleno en NLP. Es muy chevere ver que lo aprendido es de ultima actualidad. Siempre sera un reto explicar un tema muy nuevo. Gracias Profesor Alex.